# Composite mean RWP

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cmocean
import os

from dask.distributed import Client
from wave_util import construct_rolling_dataset, remove_climatology, compute_spectra, centroid, integral
from composite_util import composite_dates, two_sample_bootstrap
from hilbert_wrapper import _hilbert, _finite_difference

work = os.environ.get('WORK')+'/'
plt.rcParams.update({'font.size': 14})

In [ ]:
client = Client()

client

In [ ]:
client.close()

## Computation

In [ ]:
def envelope_phase(da):
    '''
        Absolute value and phase angle of the complex signal
    '''
    sig = xr.apply_ufunc(_hilbert,
                         *(da,len(da.lon)),
                         input_core_dims=[['lon'],[]],
                         output_core_dims=[['lon']],
                         dask='parallelized',
                         output_dtypes=[np.dtype('complex128')]
                        )

    env = np.abs(sig)
    phase = np.arctan2(np.imag(sig),np.real(sig))
    
    return xr.Dataset(dict(env=env,phase=phase))


def wavenum_speed(phase):
    '''
        Use finite differences to estimate wavenumber and phase speed
    '''
    # radians per time step
    freq = xr.apply_ufunc(_finite_difference,
                          phase,
                          input_core_dims=[['time']],
                          output_core_dims=[['time']],
                          dask='parallelized',
                          output_dtypes=[np.double]
                         )
    
    # radians per grid spacing
    wavenum = xr.apply_ufunc(_finite_difference,
                             phase,
                             input_core_dims=[['lon']],
                             output_core_dims=[['lon']],
                             dask='parallelized',
                             output_dtypes=[np.double]
                            )
    
    # grid spacing per time step
    speed = freq / wavenum
    # grid spacing per second
    speed = speed / (-6*3600)
    # meter per second
    speed = speed * (2*np.pi*6371000) * np.cos(np.radians(speed['lat'])) / (len(speed['lon'])-1)
    
    # cycles per circumference
    wavenum = wavenum / (2*np.pi) * len(wavenum['lon'])
    
    return xr.Dataset(dict(wavenumber=wavenum,phase_speed=speed))

In [ ]:
spectra = xr.open_dataarray(work+'wolfgang/spectra_30days_65-35N_wave5-8.nc')
spectra

In [ ]:
### Meridional wind

directory = work+'DATA/ERA5/eth/plev/'

files = [directory + f for f in os.listdir(directory) if f.startswith('era5_an_vwind_reg2_6h') and 
                                            (f.endswith('06.nc') or
                                             f.endswith('07.nc') or
                                             f.endswith('08.nc'))]
files.sort() 

n_valid_years = int(len(files)/3)

files = [files[i:i+3] for i in range(0,n_valid_years*3,3)]

selection = dict(lat=slice(90,20),plev=25000)

rolling = construct_rolling_dataset(files,selection=selection,n_per_window=30*4)

anomalies = remove_climatology(rolling)['var132']

anomalies['rolling'] = anomalies['rolling'] + np.timedelta64(3,'h')

In [ ]:
slow = []
fast = []

# create time series for composite construction
for w in range(5,9):                       
    
    bound = centroid(spectra.sel(wavenumber=w).mean('rolling'))
    slow.append(integral(spectra.sel(wavenumber=w),bound.values,-30))
    fast.append(integral(spectra.sel(wavenumber=w),30,bound.values))   

slow = sum(slow)
fast = sum(fast)

# select composite
dates = composite_dates(slow.rename(rolling='time'),percentile=0.9)
slow = anomalies.sel(rolling=dates.rename(time='rolling')).drop('rolling')

dates = composite_dates(fast.rename(rolling='time'),percentile=0.9)
fast = anomalies.sel(rolling=dates.rename(time='rolling')).drop('rolling')

composite = xr.concat([slow.assign_coords(speed='slow'),fast.assign_coords(speed='fast')],dim='speed')
composite = composite.compute()

In [ ]:
# Diagnostics
transform = envelope_phase(composite.chunk(dict(rolling=1))).persist()
diag = wavenum_speed(transform['phase']).compute()

# Masking & time average
envelope = transform['env'].mean('time').compute()
speed = diag['phase_speed'].where((diag['phase_speed'] > -30) * (diag['phase_speed'] < 50)).mean('time')

speed

In [ ]:
# climatology
transform = envelope_phase(anomalies.chunk(dict(rolling=1))).persist()
diag = wavenum_speed(transform['phase']).compute()

envelope_clim = transform['env'].mean('time').compute()
speed_clim = diag['phase_speed'].where((diag['phase_speed'] > -30) * (diag['phase_speed'] < 50)).mean('time')

In [ ]:
# estimate significance of composite anomalies
sig_envelope_fast = two_sample_bootstrap(envelope.sel(speed='fast'),envelope_clim,dim='rolling',nrandom=10000)
sig_speed_fast = two_sample_bootstrap(speed.sel(speed='fast'),speed_clim,dim='rolling',nrandom=10000)

sig_envelope_slow = two_sample_bootstrap(envelope.sel(speed='slow'),envelope_clim,dim='rolling',nrandom=10000)
sig_speed_slow = two_sample_bootstrap(speed.sel(speed='slow'),speed_clim,dim='rolling',nrandom=10000)

## Figure 3

In [ ]:
# plotting

fig = plt.figure(figsize=(8,6))

# climatology

ax1 = fig.add_subplot(3,2,1,projection=ccrs.EqualEarth())

envelope_clim.mean('rolling').plot(ax=ax1,transform=ccrs.PlateCarree(),
                                           levels=np.arange(10,24,2),cmap=cmocean.cm.matter,extend='both',
                                                cbar_kwargs=dict(orientation='horizontal',label=''))

ax1.set_extent([0,360,20,90],ccrs.PlateCarree())
ax1.coastlines()
ax1.set_title('Envelope')

t=ax1.text(-0.05,0.5,'Climatology',weight='bold',fontsize=12,
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax1.transAxes)

ax2 = fig.add_subplot(3,2,2,projection=ccrs.EqualEarth())

speed_clim.mean('rolling').plot(ax=ax2,transform=ccrs.PlateCarree(),
                                             levels=np.arange(0,7),cmap=cmocean.cm.rain,extend='both',
                                                cbar_kwargs=dict(orientation='horizontal',label=''))

ax2.set_extent([0,360,20,90],ccrs.PlateCarree())
ax2.coastlines()
ax2.set_title('Phase speed')


# envelope anomalies

ax3 = fig.add_subplot(3,2,3,projection=ccrs.EqualEarth())

(envelope.sel(speed='slow').mean('rolling')-envelope_clim.mean('rolling')).plot(ax=ax3,transform=ccrs.PlateCarree(),
                                                                                            levels=np.linspace(-3,3,7),cmap=cmocean.cm.delta,extend='both',
                                                                                add_colorbar=False)

ax3.contourf(sig_envelope_slow['lon'],sig_envelope_slow['lat'],sig_envelope_slow.astype(np.double),
             transform=ccrs.PlateCarree(),
             levels=[0,0.5,1],hatches=['','..'],
             alpha=0)

ax3.set_extent([0,360,20,90],ccrs.PlateCarree())
ax3.coastlines()
ax3.set_title('')

ax3.text(-0.05,0.6,'Amplified Slow',weight='bold',fontsize=12,
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax3.transAxes)


ax4 = fig.add_subplot(3,2,5,projection=ccrs.EqualEarth())

C1 = (envelope.sel(speed='fast').mean('rolling')-envelope_clim.mean('rolling')).plot(ax=ax4,transform=ccrs.PlateCarree(),
                                                                                            levels=np.linspace(-3,3,7),cmap=cmocean.cm.delta,extend='both',
                                                                                add_colorbar=False)

ax4.contourf(sig_envelope_fast['lon'],sig_envelope_fast['lat'],sig_envelope_fast.astype(np.double),
             transform=ccrs.PlateCarree(),
             levels=[0,0.5,1],hatches=['','..'],
             alpha=0)

ax4.set_extent([0,360,20,90],ccrs.PlateCarree())
ax4.coastlines()
ax4.set_title('')

ax4.text(-0.05,0.4,'Amplfied Fast',weight='bold',fontsize=12,
         va='bottom', ha='center',
         rotation='vertical', rotation_mode='anchor',
         transform=ax4.transAxes)



# speed anomalies

ax5 = fig.add_subplot(3,2,4,projection=ccrs.EqualEarth())

(speed.sel(speed='slow').mean('rolling')-speed_clim.mean('rolling')).plot(ax=ax5,transform=ccrs.PlateCarree(),
                                                                                       levels=np.linspace(-2,2,9),cmap=cmocean.cm.diff,extend='both',
                                                                          add_colorbar=False)

ax5.contourf(sig_speed_slow['lon'],sig_speed_slow['lat'],sig_speed_slow.astype(np.double),
             transform=ccrs.PlateCarree(),
             levels=[0,0.5,1],hatches=['','..'],
             alpha=0)

ax5.set_extent([0,360,20,90],ccrs.PlateCarree())
ax5.coastlines()
ax5.set_title('')


ax6 = fig.add_subplot(3,2,6,projection=ccrs.EqualEarth())

C2 = (speed.sel(speed='fast').mean('rolling')-speed_clim.mean('rolling')).plot(ax=ax6,transform=ccrs.PlateCarree(),
                                                                                       levels=np.linspace(-2,2,9),cmap=cmocean.cm.diff,extend='both',
                                                                          add_colorbar=False)

ax6.contourf(sig_speed_fast['lon'],sig_speed_fast['lat'],sig_speed_fast.astype(np.double),
             transform=ccrs.PlateCarree(),
             levels=[0,0.5,1],hatches=['','..'],
             alpha=0)

ax6.set_extent([0,360,20,90],ccrs.PlateCarree())
ax6.coastlines()
ax6.set_title('')


# config figure
fig.subplots_adjust(0,0,1,1,0.1,0.1)

plt.colorbar(C1,ax=[ax3,ax4],orientation='horizontal')

cbar = plt.colorbar(C2,ax=[ax5,ax6],orientation='horizontal')
cbar.ax.set_xticks([-2,-1,0,1,2])
